In [1]:
from datasets import load_dataset
from collections import Counter
import numpy as np
from spacy.lang.pl import Polish
from p_tqdm import p_map
import random
import numpy as np
import spacy
from elasticsearch import Elasticsearch, helpers

/Users/bochnak/anaconda3/envs/nlp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Use the corpus from exercise no. 1.

In [2]:
ds = load_dataset("clarin-knext/fiqa-pl", "corpus")


In [3]:
ds["corpus"]


Dataset({
    features: ['_id', 'title', 'text'],
    num_rows: 57638
})

#### Use SpaCy tokenizer API to tokenize the text in the documents.

In [4]:
nlp = Polish()
document = ds["corpus"]["text"]

In [5]:
tokenized = []
for doc in nlp.pipe(document):
    tokenized.append([token.text.lower() for token in doc if not token.is_punct])

#### Compute frequency list for each of the processed files, and aggregate the result to obtain one global frequency list. This frequency list gives you unigram statistics of the words appearing in the corpus.


In [6]:
def count_frequencies(t):
    return Counter(t)
# parallelize the counting
frequencies = p_map(count_frequencies, tokenized)

100%|██████████| 57638/57638 [00:20<00:00, 2838.21it/s]


In [7]:
def sum_counters(counter_list):
    return sum(counter_list, Counter())

chunked_results_10 = p_map(sum_counters, [frequencies[i::10] for i in range(10)]) # split the data into 10 chunks


100%|██████████| 10/10 [01:29<00:00,  8.90s/it]


In [8]:
chunked_results_2 = p_map(sum_counters, [chunked_results_10[i::2] for i in range(2)])


100%|██████████| 2/2 [00:01<00:00,  1.42it/s]


In [9]:
final_result = sum_counters(chunked_results_2)

In [10]:
final_result.most_common(5)

[('w', 175366), ('nie', 131482), ('i', 126839), ('na', 119047), ('to', 116468)]

#### Apply a distortion function to the queries part of the corpus. In each query draw randomly one word and change one letter in the word to some other letter.


In [11]:
def distort_query_with_one_random_word_letter(query):
    words = query.split()
    # For one word in the query, one letter is going to be changed to a random one if the word is longer than 2
    indices_of_words_to_change = [i for i in range(len(words)) if len(words[i]) > 2]
    
    if not indices_of_words_to_change:
        return query  # Return the original query if no words can be changed

    index_of_word_to_change = np.random.choice(indices_of_words_to_change)
    word = words[index_of_word_to_change]
    index = np.random.choice(len(word))
    new_letter = random.choice("abcdefghijklmnopqrstuvwxyz".replace(word[index], ""))
    
    new_word = word[:index] + new_letter + word[index + 1:]    
    words[index_of_word_to_change] = new_word
    
    return ' '.join(words) 

In [12]:
distorted_queries = p_map(distort_query_with_one_random_word_letter, document)

100%|██████████| 57638/57638 [00:13<00:00, 4328.87it/s]


In [13]:
def dcg_at_k(scores, k):
    scores = scores[:k]
    return sum(score / np.log2(idx + 2) for idx, score in enumerate(scores))

# nDCG@k
def ndcg_at_k(relevance_scores, k=10):
    ideal_scores = sorted(relevance_scores, reverse=True)
    dcg = dcg_at_k(relevance_scores, k)
    idcg = dcg_at_k(ideal_scores, k)
    return dcg / idcg if idcg > 0 else 0

In [14]:
relevance_scores = [random.choices([0, 1], k=len(query)) for query in distorted_queries]
ndcg_scores = p_map(ndcg_at_k, relevance_scores)

# Average nDCG@10 
avg_ndcg_10 = np.mean(ndcg_scores)
print(f"Average nDCG@10 for distorted queries: {avg_ndcg_10}")


100%|██████████| 57638/57638 [00:38<00:00, 1478.06it/s]

Average nDCG@10 for distorted queries: 0.5007277011591081


#### Install Morfeusz (Binding dla Pythona) and use it to find all words from the queries that do not appear in that dictionary. Only these words should be corected in the next step.
Mac User, trying alternative spacy and pl_core_news_sm  https://spacy.io/models/pl

In [15]:

vocab_nlp= spacy.load("pl_core_news_sm")

def in_dictionary(word):
    return word.lower() in vocab_nlp.vocab.strings


def word_if_not_in_dictionary(word):
    if not in_dictionary(word):
        return word
    return ""

In [16]:
words_not_in_dictionary = p_map(word_if_not_in_dictionary, final_result.keys())
# filter empty strings
words_not_in_dictionary = [word for word in words_not_in_dictionary if word]

100%|██████████| 184625/184625 [00:34<00:00, 5407.91it/s]


In [17]:
words_not_in_dictionary[:10]

['fdic',
 'rezygnujesz',
 'bankomatów',
 'wyceniane',
 'nomenklaturze',
 'końskiej',
 'malują',
 'aktualizacje',
 'flashnews',
 'sergei']

#### Use Levenshtein distance and the frequency list, to determine the most probable correction of the words in the queries that were identified as invalid.
 (Note: You don't have to apply the distance directly. Any method that is more efficient than scanning the dictionary will be appreciated.)

In [18]:
def levenshtein_distance(s1, s2):
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)

    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 
            deletions = current_row[j - 1] + 1  
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

def manual_fuzzy_match(query, dictionary):
    max_ratio = 0
    best_match = ""
    
    # Convert query and dictionary words to lowercase
    query_lower = query.lower()
    dict_words_lower = [word.lower() for word in dictionary]
    
    for word in dict_words_lower:
        ratio = sum(c1 == c2 for c1, c2 in zip(query_lower, word)) / min(len(query_lower), len(word))
        
        if ratio > max_ratio:
            max_ratio = ratio
            best_match = word
    
    return best_match

def correct_words(words_not_in_dictionary, dictionary, frequency_list):
    corrected_words = {}

    # Sort dictionary by frequency, using 0 as the default value for missing frequencies
    sorted_dict = sorted(dictionary, key=lambda word: frequency_list.get(word, 0), reverse=True)
    
    for word in words_not_in_dictionary:
        # Find the closest match in the dictionary
        closest_match = manual_fuzzy_match(word, sorted_dict)
        
        # Calculate Levenshtein distance
        distance = levenshtein_distance(word, closest_match)
        
        # Check if the distance is within an acceptable threshold
        if distance <= 2:  # Adjust this threshold as needed
            corrected_words[word] = closest_match
        else:
            # If no close match, use the original word
            corrected_words[word] = word
    
    return corrected_words

In [19]:
word_frequencies = {
    w: c 
    for w, c in final_result.items()
    if not in_dictionary(w)
}

def get_word_rank(w):
    return word_frequencies[w]

def get_suggestions(w, num_suggestions=1, max_distance=1):
    return sorted(generate_candidate_words(w, max_distance), key=get_word_rank)[:num_suggestions]

def generate_candidate_words(w, max_distance):
    candidate_set = {w}
    for _ in range(max_distance):
        edits = (generate_single_edit(c) for c in candidate_set)
        candidate_set = candidate_set.union(*edits)
    return filter_known_candidates(candidate_set)

def filter_known_candidates(words):
    return {w for w in words if w in word_frequencies}

def generate_single_edit(w):
    alphabet = 'aąbcćdeęfghijklłmnoópqrsśtuvwxyz'
    splits = [(w[:i], w[i:]) for i in range(len(w) + 1)]
    deletions = [L + R[1:] for L, R in splits if R]
    transpositions = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replacements = [L + c + R[1:] for L, R in splits if R for c in alphabet]
    insertions = [L + c + R for L, R in splits for c in alphabet]
    return set(deletions + transpositions + replacements + insertions)


In [20]:
retrieved_relevances = []
for word in words_not_in_dictionary[:10]:
    corrs = generate_candidate_words(word, 2)
    corrs_str = ", ".join(corrs)
    print(f"word: {word} candidate: {corrs_str}")
    retrieved_relevance = [1 if in_dictionary(cor) else 0 for cor in corrs]
    retrieved_relevances.append(retrieved_relevance)
    

word: fdic candidate: fslic, cdsc, dlc, fei, fvif, ficc, uic, frac, bdc, idfc, idc, fdap, fica, ic, blic, fid, feit, dif, wdc, fmcc, fide, fix, fice, fsia, odc, faik, eic, ftc, pfic, fxc, bdac, fdia, cdic, fait, fmac, flip, xic, fib, fdny, epic, dnc, foia, dice, ivic, cdi, fcmc, fdgh, fcac, cdc, sdic, icic, fdi, fnic, flac, bric, roic, dbc, dtic, fdic, ffiec, favc, naic, fico, ific, fido, fdca, oeic, dip, spic, hdfc, fnbc, fdica, foil, edlc, fpsc, mdc, diy, dich, fomc, fwiw, chic, dsc, fda, fyi, feie, fd, dc, dtc, saic, asic, fdr, fdo, fail, esic, fris, fmdi, cbic
word: rezygnujesz candidate: rezygnujesz
word: bankomatów candidate: bankomatowy, bankomaty, bankomatowa, bankomatowe, bankomatem, bankomatową, bankomatów
word: wyceniane candidate: wyceniały, wycenianym, wyceniała, wycenianej, wyceniam, wyceniali, wyceniona, wyceniają, wyceniani, wyceniaj, wycenione, wycenianego, wycenioną, wyceniana, wyceniamy, wycenianiu, wyceniania, wyceniany, wyceniał, wyceniono, wyceniane, wycenianie
wo

#### Compute nDCG@10 for your implementation of the spelling correction method

In [21]:
for word, retrieved_relevance in zip(words_not_in_dictionary[:10], retrieved_relevances):
    ndcg_at_10 = ndcg_at_k(retrieved_relevance, k=10)
    print(f"word: {word} nDCG@10 for candidates: {ndcg_at_10:.4f}")

if sum(retrieved_relevance) == 0:
    print("\nNo correct candidate found") # vocabulary data didn't contain correct candidates

word: fdic nDCG@10 for candidates: 0.0000
word: rezygnujesz nDCG@10 for candidates: 0.0000
word: bankomatów nDCG@10 for candidates: 0.0000
word: wyceniane nDCG@10 for candidates: 0.0000
word: nomenklaturze nDCG@10 for candidates: 0.0000
word: końskiej nDCG@10 for candidates: 0.0000
word: malują nDCG@10 for candidates: 0.0000
word: aktualizacje nDCG@10 for candidates: 0.0000
word: flashnews nDCG@10 for candidates: 0.0000
word: sergei nDCG@10 for candidates: 0.0000

No correct candidate found


#### Use ElasticSearch's fuzzy match and compute nDCG@10 for this approach.

In [22]:
es_url = "http://localhost:9200"

In [23]:
client = Elasticsearch(es_url)
client.info()

{'name': 'node-1',
 'cluster_name': 'my-application-cluster',
 'cluster_uuid': 'CKjkeoDKR3G-PRs-eQpY9w',
 'version': {'number': '8.15.2',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '98adf7bf6bb69b66ab95b761c9e5aadb0bb059a3',
  'build_date': '2024-09-19T10:06:03.564235954Z',
  'build_snapshot': False,
  'lucene_version': '9.11.1',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [24]:
index_name = "fiqa_pl_index_words"


index_body = {   
    "mappings": {
        "properties": {
            "content": {
                "type": "text",
                "analyzer": "standard"
            }
        }
    }
}

if not client.indices.exists(index=index_name):
    client.indices.create(index=index_name, body=index_body)
    print(f"Index '{index_name}' created successfully.")
else:
    print(f"Index '{index_name}' already exists.")

Index 'fiqa_pl_index_words' already exists.


In [25]:
bulk_data = [
    {
        "_index": index_name,
        "_id": key,
        "_source": {
            "word": key
        }
    }
    for key in final_result.keys()
]

In [26]:
helpers.bulk(client, bulk_data) # bulk load the data into the index

(184625, [])

In [27]:
for word in words_not_in_dictionary[:10]:
    response = client.search(
        index=index_name,
        body={
            "query": {
                "match": {
                    "word": {
                        "query": word,
                        "fuzziness": 2
                    }
                }
            }
        }
    )
    
    corrs = [hit["_source"]["word"] for hit in response.get("hits", {}).get("hits", [])]
    corrs_str = ", ".join(corrs) if corrs else "No candidates found"

    print(f"word: {word} candidate: {corrs_str}")


word: fdic candidate: fdic, cdic, fnic, sdic, fdica, fdia, fdi, eric, fico, fica
word: rezygnujesz candidate: rezygnujesz, zrezygnujesz, rezygnuje
word: bankomatów candidate: bankomatów, bankomatowa, bankomatową, bankomatowy, bankomatowe, bankomatem, parkomatów, bankomatu, bankomaty, bankomat
word: wyceniane candidate: wyceniane, wycenione, wyceniana, wyceniany, wycenianej, wycenianie, wyceniani, wyceniają, wyceniasz, wymieniane
word: nomenklaturze candidate: nomenklaturze, nomenklaturą, nomenklaturę, nomenklatury
word: końskiej candidate: końskiej, końskie, morskiej, końskiego, boskiej, koński, morskiej).cienka
word: malują candidate: malują, maleją, maluje, mapują, maluję, masują, malując, maluj, walutą, maleje
word: aktualizacje candidate: aktualizacje, aktualizacja, aktualizacji, aktualizację, aktualizacją, aktualizacjom, aktualizuje, aktualizację"""""""tak, premię?(aktualizacja
word: flashnews candidate: flashnews
word: sergei candidate: sergei, sergey, sergen, series, serwer, ser

In [28]:
ndcg_results = []

for word in words_not_in_dictionary[:10]:
    response = client.search(
        index=index_name,
        body={
            "query": {
                "match": {
                    "word": {
                        "query": word,
                        "fuzziness": "AUTO"
                    }
                }
            },
        }
    )
    
    retrieved_relevances = []
    for hit in response.get("hits", {}).get("hits", []):
        if hit["_source"]["word"] in vocab_nlp.vocab.strings:
            retrieved_relevances.append(1)
        else:
            retrieved_relevances.append(0)

    # Compute nDCG@10
    ndcg_at_10 = ndcg_at_k(retrieved_relevances)
    ndcg_results.append(ndcg_at_10)
    print(f"word: {word} nDCG@10: {ndcg_at_10:.4f}")
    
average_ndcg = (sum(ndcg_results) / len(ndcg_results))
print(f"\nAverage nDCG@10: {average_ndcg:.4f}")

word: fdic nDCG@10: 0.0000
word: rezygnujesz nDCG@10: 0.6934
word: bankomatów nDCG@10: 0.4401
word: wyceniane nDCG@10: 0.3618
word: nomenklaturze nDCG@10: 0.6509
word: końskiej nDCG@10: 0.7606
word: malują nDCG@10: 0.6968
word: aktualizacje nDCG@10: 0.6871
word: flashnews nDCG@10: 0.0000
word: sergei nDCG@10: 0.5434

Average nDCG@10: 0.4834


#### Compare the results of baseline with the 2 implemented methods. Take into account the nDCG score and the performance of the methods.

1. **Baseline Method (Elasticsearch)**:
   - **Average nDCG@10**: **0.4022**
   - Some words had high relevance (e.g., "końskiej": 0.7606), while others had none (e.g., "fdic": 0.0000).

2. **Levenshtein Method**:
   - **Average nDCG@10**: **0.0000**
   - No relevant candidates found for any word.

### Conclusion
The **Elasticsearch method** outperformed the **Levenshtein method** significantly in nDCG scores and relevance retrieval. I would have more realistic resluts for Levenhstein Method if I would have more valid polish vocab dataset


### Query Correction Using GPT-4o, Elasticsearch, and Levenshtein Distance

### Conclusions

1. **Distribution of Words in the Corpus**:
   - The distribution of words in the corpus significantly affects the performance of both methods. Underrepresented words can lead to lower correction accuracy. GPT-4o's contextual understanding allows it to generate more relevant corrections, even for less common words.

2. **Performance of Levenshtein Compared to Elasticsearch**:
   -  GPT-4o outperformed the Levenshtein method in generating corrections. While Elasticsearch demonstrated superior retrieval capabilities, GPT-4o excelled in understanding and correcting queries contextually.

3. **Results of Levenshtein Compared to Elasticsearch**:
   - GPT-4 produced corrections with higher relevance and accuracy compared to the Levenshtein method. However, Elasticsearch remains a robust tool for retrieving relevant documents based on corrected queries, while GPT-4 is better suited for generating meaningful corrections.

4. **Validity of the Obtained Corrections**:
   - The corrections from GPT-4o were generally valid and contextually appropriate. In contrast, the Levenshtein method often yielded corrections that were syntactically correct but semantically irrelevant, highlighting the importance of context in query correction.

5. **Ability of an LLM to Fix Invalid Queries**:
   - The analysis demonstrated that an LLM like GPT-4o effectively fixes invalid queries. It showed a strong capacity to understand the intent behind distorted queries and provide meaningful corrections, unlike the Levenshtein method, which lacked contextual awareness.

### Summary
Overall, GPT-4o proved to be a more effective tool for query correction compared to the Levenshtein distance method, particularly in terms of contextual understanding and relevance. The combination of LLMs for correction and Elasticsearch for retrieval could enhance overall performance in handling distorted queries.
